# Behavioral Cloning


In [1]:
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import csv
import cv2
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import *
from keras.models import *
from keras.optimizers import Adam
from time import time
from random import randint

%matplotlib inline
print('Import successful.')

Using TensorFlow backend.


Import successful.


Load the data and get an impression of the images

In [2]:
source_path = 'data_middle/driving_log.csv'

logs = []
reader = csv.reader(open(source_path, 'rt'))
for line in reader:
    logs.append(line)

print('logs created succesfully')

logs created succesfully


In [3]:
# get rid of the first line that does not contain information about image paths and the corresponding information
print('logs.pop[0]: ', logs.pop(0))
print('len(logs[0]): ', len(logs[0]))
print('logs[0]: ', logs[0])
print('logs[1][0]: ', logs[1][0])

logs.pop[0]:  ['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed']
len(logs[0]):  7
logs[0]:  ['IMG/center_2016_12_01_13_30_48_287.jpg', ' IMG/left_2016_12_01_13_30_48_287.jpg', ' IMG/right_2016_12_01_13_30_48_287.jpg', ' 0', ' 0', ' 0', ' 22.14829']
logs[1][0]:  IMG/center_2016_12_01_13_30_48_404.jpg


Load all the data in appropriate lists

In [4]:
def resize_image(input_image):
    '''@brief Reduce image size for computational reasons.
    '''
    x_size = 64 
    y_size = 32
    return cv2.resize(input_image, (x_size, y_size))

def crop_image(input_image):
    '''@brief Cut off the sky and lower part of the picture since it contains
              little information about the track.
    '''
    offset_low = 40
    offset_high = 140
    return input_image[offset_low:offset_high]

def change_color_space(input_image):
    ''' Return image in S dimension of HSV color space
    '''
    return  cv2.cvtColor(input_image, cv2.COLOR_RGB2HSV)[:, :, 1]

def image_wrapper(input_image):
    '''@brief summarize all relevant image processing functions
    '''
    return change_color_space(resize_image(input_image))

def img_load(path_to_table, steering_offset):
    #read the path to the images
    reader = csv.reader(open(path_to_table, 'rt'))
    new_logs = []
    for line in reader:
        new_logs.append(line) #save the path in a list
    #delete first line because it contains unnecessary information
    new_logs.pop(0)
    X = [] # empty list of "images"
    y = [] # empty list of steering angles
    folder = 'data/'
    steeringThresh = 0.1 # Ignore images that have a steering angle below that threshold
    counter = 0 # counts the accepted images
    total_counter = 0 # counts all the images
    for center, left, right, steering, throttle, brake, speed in new_logs:
        total_counter += 1
        # avoid importing data that have very small steering angles with a probability of 70%
        if (float(steering) < steeringThresh) and (randint(0, 100) > 70):
            counter += 1
            # preprocess the images and save them in a list
            X.append(image_wrapper(plt.imread(folder + center))) # include preprocessing
            # append left and right images    
            X.append(image_wrapper(plt.imread(folder + left[1:]))) # include preprocessing
            X.append(image_wrapper(plt.imread(folder + right[1:]))) # include preprocessing

            #get the steering angle
            y.append(float(steering))
            #add the steering offset to the left and right images
            y.append(float(steering) + steering_offset)
            y.append(float(steering) - steering_offset)
        else: #accept all images that have a steering angle larger than the threshold
            # preprocess the images and save them in a list
            X.append(image_wrapper(plt.imread(folder + center))) # include preprocessing
            # append left and right images    
            X.append(image_wrapper(plt.imread(folder + left[1:]))) # include preprocessing
            X.append(image_wrapper(plt.imread(folder + right[1:]))) # include preprocessing

            #get the steering angle
            y.append(float(steering))
            #add the steering offset to the left and right images
            y.append(float(steering) + steering_offset)
            y.append(float(steering) - steering_offset)
            
    print('Neglected images [n]: ', total_counter - counter)
    print('Neglected images [%]: ', (total_counter - counter) / total_counter)
    
    return X, y


In [5]:
start_time = time()
X_train, y_train = img_load(source_path, float(25/100))
print('Image loading time: ', time() - start_time, 's')
print('len(X_train): ', len(X_train))
print('len(y_train): ', len(y_train))

Neglected images [n]:  6930
Neglected images [%]:  0.7505686125852918
Image loading time:  104.54295563697815 s
len(X_train):  27699
len(y_train):  27699


Flip images around the y-axis

In [6]:
#flip the images around the y-axis
X_flipped = []
y_flipped = []
for X in X_train:
    X_flipped.append(np.fliplr(X))
# adjust the steering by changing the foresign
for y in y_train:
    y_flipped.append(-y)
    
# merge flipped data with original data
X_total = X_train + X_flipped
y_total = y_train + y_flipped

print('len(X_total): ', len(X_total))
print('len(y_total): ', len(y_total))

len(X_total):  55398
len(y_total):  55398


In [7]:
X_np = np.array(X_total)
y_np = np.array(y_total)
print('X_np.shape: ', X_np.shape)
print('y_np.shape: ', y_np.shape)

X_np.shape:  (55398, 32, 64)
y_np.shape:  (55398,)


In [8]:
# Shuffle the data
X_np, y_np = shuffle(X_np, y_np)
X_np = np.expand_dims(X_np, axis=3)
image_size = np.shape(np.array(X_np[0]))
print('image_size: ', image_size)
print('X_np[0]: ', X_np[0])

image_size:  (32, 64, 1)
X_np[0]:  [[[255]
  [ 64]
  [ 43]
  ..., 
  [ 96]
  [ 95]
  [ 95]]

 [[ 49]
  [103]
  [ 40]
  ..., 
  [ 94]
  [ 94]
  [ 94]]

 [[230]
  [152]
  [ 47]
  ..., 
  [ 92]
  [ 92]
  [ 92]]

 ..., 
 [[ 30]
  [ 31]
  [ 27]
  ..., 
  [ 81]
  [ 81]
  [ 80]]

 [[ 29]
  [ 23]
  [ 34]
  ..., 
  [255]
  [255]
  [ 12]]

 [[ 40]
  [ 36]
  [ 32]
  ..., 
  [ 71]
  [ 35]
  [ 38]]]


Now that the image preproccessing finished, the neural net needs to be defined in Keras. 

In [9]:
model = Sequential()

model.add(Lambda(lambda x: x/255.0 - 0.5,input_shape=(32, 64, 1)))

model.add(Conv2D(3, 6, 6, border_mode='valid', activation='relu')) #introducing non-linearity
model.add(MaxPooling2D((3,3), (3,3), 'valid'))
model.add(Conv2D(3, 3, 3, border_mode='valid', activation='relu'))
model.add(MaxPooling2D((3,3), (3,3), 'valid'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1)) 

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 32, 64, 1)     0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 27, 59, 3)     111         lambda_1[0][0]                   
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 9, 19, 3)      0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 7, 17, 3)      84          maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [10]:
# keras model compile, choose optimizer and loss func
start_time = time()
# train the model
model.compile(loss = 'mse', optimizer = 'adam')
model.fit(x = X_np, y = y_np, validation_split = 0.2, shuffle = False, nb_epoch=5) #data is shuffled above
print('Total training time: ', time() - start_time)
model.save('model_test.h5')

Train on 44318 samples, validate on 11080 samples
Epoch 1/5
44318/44318 [==============================] - 119s - loss: 0.0479 - val_loss: 0.0345
Epoch 2/5
44318/44318 [==============================] - 121s - loss: 0.0375 - val_loss: 0.0311
Epoch 3/5
44318/44318 [==============================] - 130s - loss: 0.0350 - val_loss: 0.0304
Epoch 4/5
44318/44318 [==============================] - 129s - loss: 0.0335 - val_loss: 0.0292
Epoch 5/5
44318/44318 [==============================] - 153s - loss: 0.0326 - val_loss: 0.0289
Total training time:  654.0578637123108
